In [10]:
import pandas as pd

In [11]:
url = 'https://drive.google.com/file/d/1DlYzfiLxAVE4LGGar15gjR7gUG5UJ2K6' #@param {type:"string"}
print(f'The address of your .csv is: {url}')

The address of your .csv is: https://drive.google.com/file/d/1DlYzfiLxAVE4LGGar15gjR7gUG5UJ2K6


In [12]:
#Read the csv
path = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-1]
df_original = pd.read_csv(path)

In [13]:
df_original

,id,pack_size_text,pack_size,item_volume,item_density,item_weight,total_weight
0,1,NaN,20.0,NaN,NaN,3.0,60.0
1,2,5 bottles of 1 kg,5.0,NaN,NaN,1.0,5.0
2,3,NaN,1.0,2.0,0.9,1.8,1.8
3,4,6 boxes of 3 kg,NaN,NaN,NaN,NaN,NaN
4,5,NaN,5.0,3.4,2.0,NaN,NaN
5,6,8 packs of 4 kg,NaN,NaN,NaN,NaN,32.0
6,7,6 boxes of 4 kg,NaN,NaN,NaN,NaN,NaN
7,8,4 boxes of 3 kg,4.0,NaN,NaN,3.0,NaN
8,9,NaN,3.0,NaN,NaN,1.0,NaN
9,10,NaN,2.0,3.0,4.0,NaN,NaN


In [14]:
#Set the function

def parse_item_weight(df: pd.DataFrame):

  if not set(['id', 'pack_size_text', 'pack_size', 'item_volume', 'item_density',
       'item_weight', 'total_weight']).issubset(set(df.columns)):
        raise ValueError("DataFrame is missing one or more required columns.\n The required columns are: 'id', 'pack_size_text', 'pack_size', 'item_volume', 'item_density','item_weight', 'total_weight'")
  
  #Set item_weight column
  df_check = df['item_weight'].isnull()
  df_null = df.loc[df_check]

  for item in df_null.iterrows():
    id = item[1]['id']
    #Get the value from pack_size_text column
    pack_size_value_1st = float(item[1]['pack_size_text'].split(' ')[0]) if type(item[1]['pack_size_text'])!=float else pd.np.nan
    pack_size_value_2nd = float(item[1]['pack_size_text'].split(' ')[-2]) if type(item[1]['pack_size_text'])!=float else pd.np.nan
    #print(pack_size_value_1st)

    #Get the value by multiplying the columns 'item_volume' and 'item_density'
    math_value_1 = float(item[1]['item_volume'])*float(item[1]['item_density'])

    #Get the value by dividing the column 'total_weight' by 'pack_size'
    math_value_2 = float(item[1]['total_weight'])/float(item[1]['pack_size'])
    
    #Update the values 
    df_null.loc[(df_null['id']==id) & (pd.isnull(df_null['pack_size'])), 'pack_size'] = pack_size_value_1st
    df_null.loc[(df_null['id']==id), 'item_weight']= pack_size_value_2nd if not pd.isnull(pack_size_value_2nd) else math_value_1 if not pd.isnull(math_value_1) else math_value_2 if not pd.isnull(math_value_2) else pd.np.nan

  
  
  #Update the dataframe for item_weight
  df.set_index('id', inplace=True)
  df_null.set_index('id', inplace=True)
  df.update(df_null)
  df.reset_index(inplace=True)

  #Do the final calculation
  df['total_weight']=df['pack_size']*df['item_weight']

  return df

In [15]:
new_df = parse_item_weight(df_original)

new_df

<ipython-input-14-f227a6fae917>:16: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  pack_size_value_1st = float(item[1]['pack_size_text'].split(' ')[0]) if type(item[1]['pack_size_text'])!=float else pd.np.nan
<ipython-input-14-f227a6fae917>:17: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  pack_size_value_2nd = float(item[1]['pack_size_text'].split(' ')[-2]) if type(item[1]['pack_size_text'])!=float else pd.np.nan


,id,pack_size_text,pack_size,item_volume,item_density,item_weight,total_weight
0,1,NaN,20.0,NaN,NaN,3.0,60.0
1,2,5 bottles of 1 kg,5.0,NaN,NaN,1.0,5.0
2,3,NaN,1.0,2.0,0.9,1.8,1.8
3,4,6 boxes of 3 kg,6.0,NaN,NaN,3.0,18.0
4,5,NaN,5.0,3.4,2.0,6.8,34.0
5,6,8 packs of 4 kg,8.0,NaN,NaN,4.0,32.0
6,7,6 boxes of 4 kg,6.0,NaN,NaN,4.0,24.0
7,8,4 boxes of 3 kg,4.0,NaN,NaN,3.0,12.0
8,9,NaN,3.0,NaN,NaN,1.0,3.0
9,10,NaN,2.0,3.0,4.0,12.0,24.0
